# Conception BDD Normalisée et Conforme au RGPD (SIDORA AI)
---
## 

## CRUD pour verifier les donneés

In [ ]:
# import
import datetime
import pandas as pd
from sqlalchemy import and_, or_

from sqlalchemy import  create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.sql import func

from components.models import Log, Client, DonnePersonnel, Commande, Produit, Genre, Promotion, Age, Region, Platform, Publisher, Year

In [7]:
#engine
engine = create_engine("sqlite:///BD_Ventes_de_jeux_video.db") 

# session
Session = sessionmaker(bind=engine)
session = Session()

## C - CREATE
- create_client(age_id, region_id)
- create_donne_personnel(login, mot_de_passe_hash)
- create_commande(client_id, produit_id, nb_produit)
- create_promotion(produit_id:int, promotion_percent:int, region_id_promo:int)



In [8]:
# Function pour creater 
def create_client(age_id:int, region_id:int):
    """Create and persist a new Client in the database.

    Args:
        age_id (int): ID of the age category for the client.
        region_id (int): ID of the region where the client belongs.

    Returns:
        Client: The newly created Client object.

    Raises:
        Exception: If the session commit fails, the transaction is rolled back and the exception is re-raised.
    """
    try:
        client = Client(
            age_id=age_id,
            region_id=region_id
        )
        session.add(client)
        session.commit()
        return client
    except Exception as e:
        session.rollback()
        raise e
    
def create_donne_personnel(login, mot_de_passe_hash):
    """Create and persist a new DonnePersonnel (personal data) record for a client.

    Args:
        login (str): Login username for the personal data.
        mot_de_passe_hash (str): Hashed password.

    Returns:
        DonnePersonnel: The newly created DonnePersonnel object.

    Raises:
        Exception: If the session commit fails, the transaction is rolled back and the exception is re-raised.
    """
    try:
        donne = DonnePersonnel(
            login=login,
            mot_de_passe_hash=mot_de_passe_hash,
            date_suppression=None,
            anonymise=False
        )
        session.add(donne)
        session.commit()
        return donne
    except Exception as e:
        session.rollback()
        raise e

def create_commande(client_id, produit_id, nb_produit):
    """Create and persist a new order (Commande) in the database, optionally applying a promotion.

    The function looks for a promotion associated with the given product — 
    if found, the order will reference the promotion; otherwise, no promotion is applied.

    Args:
        client_id (int): ID of the client placing the order.
        produit_id (int): ID of the product being ordered.
        nb_produit (int): Number of units ordered.

    Returns:
        Commande: The newly created Commande object.

    Raises:
        Exception: If the session commit fails. The session will be rolled back and the exception re-raised.
    """
    try:
        promo = session.query(Promotion).filter(Promotion.produit_id=={produit_id}).first()
        promo_id = promo.promotion_id if promo else 0
        session.add(Commande(
            client_id = client_id,
            produit_id = produit_id,
            nb_produit = nb_produit,
            promotion_id = promo_id
            ))
        session.commit()
    except Exception as e:
        session.rollback()
        raise e
    
def create_promotion(produit_id:int, promotion_percent:int, region_id_promo:int):
    """Create a promotion for a given product and link it to a region.

    Args:
        produit_id (int): ID of the product.
        promotion_percent (int): Discount percent.
        region_id_promo (int): ID of the region for this promotion.

    Raises:
        Exception: If commit fails, the session is rolled back and the exception re-raised.
    """
    try:
        region = session.query(Region).filter(Region.region_id==region_id_promo).first()

        obj = Promotion(
            produit_id = produit_id,
            promotion_percent = promotion_percent
        )
        obj.regions.append(region)
        session.add(obj)
        session.commit()
    except Exception as e:
        session.rollback()
        raise e

# R - READ
- read_table(table_class, limit=None, filter_exp=None) -- jeneral pour verifier
- read_promo(limit=None, filter_exp=None)
- read_produit(limit=None, filter_exp=None)
- read_command(limit=None, filter_exp=None)
- read_client(limit=None, filter_exp=None)

In [45]:
# Function pour faire un roquet et pour retourner DataFrame
def read_table(table_class, limit=None, filter_exp=None):
    """
    Lit une table SQLAlchemy et renvoie les résultats dans un DataFrame.

    Args:
        table_class: Modèle SQLAlchemy (ex.: Client, Commande).
        limit: Nombre maximum de lignes à retourner (optionnel).
        filter_exp: Expression SQLAlchemy pour filtrer (optionnel).

    Returns:
        DataFrame contenant le résultat de la requête.

    Raises:
        Exception: If commit fails, the session is rolled back and the exception re-raised.
    """

    try:
        query = session.query(table_class)
        
        if table_class is Client:
            query.update(
            {Client.date_derniere_utilisation: func.now()},
            synchronize_session=False
            )
            session.commit()

        if filter_exp is not None:
            query = query.filter(filter_exp)
        
        if limit is not None:
            query = query.limit(limit)
        
        df = pd.read_sql(query.statement, session.bind)
        return df
    
    except Exception as e:
        session.rollback()
        raise e


def read_promo(limit=None, filter_exp=None):
    """Interroger les promotions avec les noms des produits et les régions associées.
    Cette fonction retourne un DataFrame contenant les promotions jointes aux produits,
    ainsi qu'une chaîne formatée listant les régions pour chaque promotion.

    Args:
        limit (int, optional): Nombre maximal de lignes à retourner.
        filter_exp (expression SQLAlchemy, optional): Expression de filtrage à appliquer.

    Returns:
        tuple:
            - pandas.DataFrame: Résultats de la requête avec Promotion et Produit.name.
            - str: Chaîne formatée listant les promotions et les régions associées.

    Raises:
        Exception: Toute exception levée pendant l'exécution de la requête est capturée et réémise.
    """

    try:
        query = (session.query(
            Promotion,
            Produit.name
            )).join(Produit, Produit.produit_id == Promotion.produit_id)
        
        if filter_exp is not None:
            query = query.filter(filter_exp)
        if limit is not None:
            query = query.limit(limit)

        df = pd.read_sql(query.statement, session.bind, index_col="promotion_id")
        
        list_reg = ""
        for promo, prod_name in query:
            for reg in promo.regions:
                list_reg += f"{promo.promotion_percent}% apply to {prod_name}: region -- {reg.region_nom}\n"
        
        return df, list_reg
    
    except Exception as e:
        raise e

def read_produit(limit=None, filter_exp=None):
    """Interroger les produits avec leurs informations détaillées.

    Cette fonction retourne un DataFrame contenant les produits et les informations
    associées provenant des tables liées : année, plateforme, genre et éditeur.

    Args:
        limit (int, optional): Nombre maximal de lignes à retourner.
        filter_exp (expression SQLAlchemy, optional): Expression de filtrage à appliquer.

    Returns:
        pandas.DataFrame: Résultats de la requête avec les détails des produits.

    Raises:
        Exception: Toute exception levée pendant l'exécution de la requête est réémise.
    """

    try:
        query = (
            session.query(
                Produit.produit_id, Produit.prix, Produit.name,
                Year.year_nom.label("year_nom"),
                Platform.platform_nom.label("platform_nom"),
                Genre.genre_nom.label("genre_nom"),
                Publisher.publisher_nom.label("publisher_nom")
            )
            .join(Year, Year.year_cod == Produit.year_n)
            .join(Platform, Platform.platform_cod == Produit.platform_cod)
            .join(Genre, Genre.genre_cod == Produit.genre_cod)
            .join(Publisher, Publisher.publisher_cod == Produit.publisher_cod)
        )
        
        if filter_exp is not None:
            query = query.filter(filter_exp)
        if limit is not None:
            query = query.limit(limit)
        
        df = pd.read_sql(query.statement, session.bind)

        return df
    except Exception as e:
        raise e


def read_command(limit=None, filter_exp=None):
    """Interroger les commandes avec les informations sur le produit et la promotion.

    Cette fonction retourne un DataFrame contenant les commandes, le nombre de produits,
    le nom du produit et le pourcentage de promotion s'il existe.

    Args:
        limit (int, optional): Nombre maximal de lignes à retourner.
        filter_exp (expression SQLAlchemy, optional): Expression de filtrage à appliquer.

    Returns:
        pandas.DataFrame: Résultats de la requête avec les détails des commandes.

    Raises:
        Exception: Toute exception levée pendant l'exécution de la requête est réémise.
    """
     
    try:
        query = (session.query(
            Commande.commande_id,
            Commande.nb_produit,
            Commande.client_id,
            Produit.name.label("produit_nom"),
            Produit.prix.label("prix"),
            Promotion.promotion_percent,
            )
        .join(Produit, Produit.produit_id == Commande.produit_id)
        .outerjoin(Promotion, Promotion.produit_id == Commande.produit_id)
        )
        
        if filter_exp is not None:
            query = query.filter(filter_exp)
        if limit is not None:
            query = query.limit(limit)

        df = pd.read_sql(query.statement, session.bind)
        df["promotion_percent"] = df["promotion_percent"].fillna(0)
        df['prix total'] = df["nb_produit"] * df["prix"] * (1 - (0.01 * df["promotion_percent"]))
        return df
    
    except Exception as e:
        raise e

def read_client(limit=None, filter_exp=None):
    """
    Récupère les informations des clients avec les données associées et le nombre de commandes.

    Args:
        limit (int, optional): Nombre maximum de clients à récupérer. Par défaut, None = tous.
        filter_exp (Expression, optional): Expression SQLAlchemy pour filtrer les clients.
                                           Exemple : Client.region_id == 0

    Behavior:
        - Joint les tables Age et Region pour récupérer les informations associées.
        - Calcule le nombre de commandes par client.
        - Applique un filtre et une limite si fournis.
        - Retourne le résultat sous forme de DataFrame pandas.

    Returns:
        pandas.DataFrame: Contient les colonnes suivantes :
            - client_id
            - age_plage
            - region_nom
            - Nb_commande
    """
    try:
        query = (
            (session.query(
            Client.client_id,
            Age.age_plage,
            Region.region_nom,
            func.count(Commande.commande_id).label("Nb_commande")
            ))
            .join(Commande, Commande.client_id == Client.client_id)
            .join(Age, Age.age_id == Client.age_id)
            .join(Region, Region.region_id == Client.region_id)
            ).group_by(Client.client_id)
        
        if filter_exp is not None:
            query = query.filter(filter_exp)
        
        if limit is not None:
            query = query.limit(limit)  

        
        df = pd.read_sql(query.statement, session.bind, index_col="client_id")

        return df
    
    except Exception as e:
        raise e

def update_client_date(client_id):
    obj = session.get(Client, client_id)
    if obj is None:
        return None
    setattr(obj, "date_derniere_utilisation", datetime.datetime.now())
    session.commit()


### Verification 

In [13]:
read_table(Produit, limit=10, filter_exp=Produit.prix < 50)

,produit_id,name,prix,year_n,platform_cod,genre_cod,publisher_cod
0,1,Wii Sports,35,0,0,0,0
1,3,Mario Kart Wii,31,2,0,2,0
2,4,Wii Sports Resort,27,3,0,0,0
3,9,New Super Mario Bros. Wii,22,3,0,1,0
4,18,Grand Theft Auto: San Andreas,36,12,6,8,2
5,19,Super Mario World,28,13,7,1,0
6,20,Brain Age: Train Your Brain in Minutes a Day,30,7,3,5,0
7,26,Pokemon Ruby/Pokemon Sapphire,34,15,8,3,0
8,31,Pokémon Yellow: Special Pikachu Edition,39,18,2,3,0
9,43,Mario Kart 7,34,17,9,2,0


In [54]:
df_promo = read_promo(limit=5)
print(df_promo[0], "\n")
print(df_promo[1])


              promotion_percent  produit_id  \
promotion_id                                  
1                            70        5511   
2                            50        6512   
3                            50        6564   
4                            10        8839   
5                            60         936   

                                                 name  
promotion_id                                           
1                   Hatsune Miku: Project Diva Extend  
2             Konjiki no Gashbell!! Makai no Bookmark  
3                          Oddworld: Stranger's Wrath  
4                      Ultimate Board Game Collection  
5                                               Jak 3   

70% apply to Hatsune Miku: Project Diva Extend: region -- NA
50% apply to Konjiki no Gashbell!! Makai no Bookmark: region -- Other
50% apply to Oddworld: Stranger's Wrath: region -- JP
10% apply to Ultimate Board Game Collection: region -- EU
60% apply to Jak 3: region -- EU


In [28]:
read_produit( filter_exp=(Platform.platform_nom == "DS"), limit=5)

,produit_id,prix,name,year_nom,platform_nom,genre_nom,publisher_nom
0,16,111,Kinect Adventures!,2007.0,DS,Puzzle,Nintendo
1,24,111,Grand Theft Auto V,2010.0,DS,Simulation,Microsoft Game Studios
2,30,95,Call of Duty: Modern Warfare 3,2001.0,DS,Misc,Sony Computer Entertainment
3,32,112,Call of Duty: Black Ops,2007.0,DS,Misc,Sony Computer Entertainment
4,36,51,Call of Duty: Black Ops II,2015.0,DS,Misc,Sony Computer Entertainment


In [ ]:
read_command(filter_exp=Promotion.promotion_percent > 0)

,commande_id,nb_produit,client_id,produit_nom,prix,promotion_percent,prix total
0,2643,4,159,National Geographic Panda (US sales),81,30,226.8
1,2765,4,89,Hatsune Miku: Project Diva Extend,118,70,141.6
2,3356,4,126,Jak 3,86,60,137.6
3,3799,4,206,Tom Clancy's Ghost Recon 2,39,50,78.0


In [24]:
read_client(limit=3, filter_exp=and_(Region.region_nom == "NA", Age.age_plage == "0 - 6 ans") )

,client_id,age_plage,region_nom,Nb_commande
0,9,0 - 6 ans,NA,16
1,11,0 - 6 ans,NA,15
2,17,0 - 6 ans,NA,11


# U - UPDATE
- update_table(table_nom, data_id, **kwargs)

In [ ]:
def update_table(table_nom, data_id, **kwargs):
    """
    Met à jour les colonnes spécifiées d'un enregistrement dans une table SQLAlchemy.

    Args:
        table_nom (DeclarativeMeta): La classe SQLAlchemy représentant la table.
        data_id (int): L'identifiant de l'enregistrement à mettre à jour.
        **kwargs: Paires clé-valeur représentant les colonnes à modifier et leurs nouvelles valeurs.
                  Exemple : age_id=1, region_id=0

    Behavior spécifique:
        - Si la table est `Client`, la colonne `date_derniere_utilisation` sera mise à jour avec l'heure actuelle.

    Returns:
        None si l'objet avec `data_id` n'existe pas. Sinon, commit les changements dans la base de données.

    Exemple:
        update_table(Client, 51, age_id=1, region_id=0)
    """
    obj = session.get(table_nom, data_id)
    if not obj:
        return None
    
    for field, value in kwargs.items():
        setattr(obj, field, value)
    

    if table_nom is Client:
        obj.date_derniere_utilisation = func.now()
        
    session.commit()
    print(f"L’enregistrement dans {table_nom.__tablename__} a été renouvelé.")



### Vérifier le bon fonctionnement de UPDATE

In [141]:
read_client(filter_exp=(Client.client_id == 51))

,client_id,age_id,region_id,date_creation,date_derniere_utilisation
0,51,1,2,2025-12-01 13:31:37,2025-12-02 14:00:04


In [143]:
data = {
    'age_id': 1,
    'region_id': 0
}
update_table(Client, 51, **data)

L’enregistrement dans clients a été renouvelé.


In [144]:
read_client(filter_exp=(Client.client_id == 51))

,client_id,age_id,region_id,date_creation,date_derniere_utilisation
0,51,1,0,2025-12-01 13:31:37,2025-12-02 14:03:33


# D - DELETE
- delete_objet(table_nom, data_id)
- delete_filtre(table_nom, filter_exp)

In [ ]:
def delete_objet(table_nom, data_id):
    """
    Supprime un enregistrement spécifique d'une table SQLAlchemy.

    Args:
        table_nom (DeclarativeMeta): La classe SQLAlchemy représentant la table.
        data_id (int): L'identifiant de l'enregistrement à supprimer.

    Behavior:
        - Cherche l'objet dans la base via `session.get`.
        - Si l'objet existe, le supprime et commit la transaction.
        - Si l'objet n'existe pas, aucune suppression n'est effectuée.
        - Capture les exceptions et affiche un message d'erreur.

    Returns:
        None

    Exemple:
        delete_objet(Client, 51)
    """
    try:
        obj = session.get(table_nom, data_id)
        if obj is not None:
            session.delete(obj)
        session.commit()
    except Exception as e:
        print(e)

def delete_filtre(table_nom, filter_exp):
    """
    Supprime tous les enregistrements d'une table SQLAlchemy correspondant à un filtre donné.

    Args:
        table_nom (DeclarativeMeta): La classe SQLAlchemy représentant la table.
        filter_exp: Expression de filtre SQLAlchemy pour sélectionner les enregistrements à supprimer.
                    Exemple : Client.age_id == 1

    Behavior:
        - Crée une requête avec `session.query(table_nom).filter(filter_exp)`.
        - Supprime tous les enregistrements filtrés via `.delete()`.
        - Capture les exceptions et affiche un message d'erreur.

    Returns:
        None

    Exemple:
        delete_filtre(Client, Client.age_id == 1)
    """
    try:
        query = session.query(table_nom).filter(filter_exp)
        query.delete(synchronize_session=False)
        session.commit()

    except Exception as e:
        print(e)

### LOGGING

In [ ]:
def add_log(type_action, table_cible, client_id=None, details=None):
    """Ajoute une entrée dans la table des logs.

    Args:
        session: session SQLAlchemy utilisée pour la connexion à la base de données
        type_action (str): type d'action ("INSERT", "UPDATE", "DELETE", etc.)
        table_cible (str): nom de la table concernée par l'action
        client_id (int, optionnel): identifiant du client ou de l'utilisateur effectuant l'action
        details (str, optionnel): informations complémentaires, par exemple au format JSON

    Raises:
        Exception: en cas d'erreur lors de l'insertion dans la table logs,
                   la transaction est annulée (rollback) et l'exception est levée.
    """
    
    try:
        log_entry = Log(
            type_action=type_action,
            table_cible=table_cible,
            client_id=client_id,
            details=details
        )
        session.add(log_entry)
        session.commit()
    except Exception as e:
        session.rollback()
        raise e


# CLOSE ALL

In [ ]:
# session.rollback()

In [55]:
session.close()
engine.dispose()